
<div style="border-radius: 5px; background-color: #A478B8">
    <h3 style="color: white; padding: 1rem">Introduction</h1>
</div>

Bu projeye başlamadan **[Dashboard](https://developer.spotify.com/dashboard)** oluşturmamız gerekmektedir.

Dashboardunuza bir isim verin(benim verdiğim isim **recommendation** )



Dashboard > **recommendation** > systemSettings > Basic Information

CLIENT_ID  ve CLIENT_SECRET  değerlerinize ulaşın 

In [1]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = 'your_client_id'
CLIENT_SECRET = 'your_client_secret'

In [2]:
# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())



In [3]:
# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [4]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Spotipy'yi erişim belirteci ile yapılandır
    sp = spotipy.Spotify(auth=access_token)

    # Playlist'ten parçaları al
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # İlgili bilgileri çıkar ve bir sözlük listesine sakla
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Parça için ses özelliklerini al
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Albümün yayın tarihini al
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Parçanın popülerliğini al
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Parça verilerine ek bilgileri ekle
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [5]:
# Spotify çalma listesi kimliği
playlist_id = '3l7ZCv5aGefJeHOW1FePJJ'

# Çalma listesinden müzik verilerini almak için oluşturulan fonksiyonu çağır ve verileri bir DataFrame'e depola
music_df = get_trending_playlist_data(playlist_id, access_token)


In [6]:
# Display the DataFrame
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Kumralım,Yaşar,Divane,07yiI9ScrZV7DXiObRz6Sp,3tCCD2qQLdC99YkRwGPSsx,68,1996,210840,False,https://open.spotify.com/track/3tCCD2qQLdC99Yk...,...,0.942,9,-7.141,0,0.1690,0.4350,0.000000,0.1510,0.660,114.943
1,Dansöz,Serdar Ortaç,Mesafe,1bp7hCG7HxaJ0kUjD3eykN,5iKzAnS1BY583HpwxHIz1m,0,2006-04-25,316320,False,https://open.spotify.com/track/5iKzAnS1BY583Hp...,...,0.925,6,-6.133,0,0.0441,0.0149,0.000035,0.3380,0.816,129.961
2,Sor,Serdar Ortaç,Mesafe,1bp7hCG7HxaJ0kUjD3eykN,5Uzxouh00sQqfcUd6I4BnW,0,2006-04-25,235213,False,https://open.spotify.com/track/5Uzxouh00sQqfcU...,...,0.669,6,-6.372,1,0.0649,0.1540,0.000000,0.0448,0.610,100.038
3,Ben Adam Olmam,Serdar Ortaç,Yaz Yağmuru,4NRcuQwDlzuqpCRJkLy7WT,4hT4tcqFWGR4Da5ONBSMkW,55,1996-03-01,283768,False,https://open.spotify.com/track/4hT4tcqFWGR4Da5...,...,0.724,2,-13.405,0,0.0486,0.2000,0.001230,0.0363,0.891,98.021
4,Hovarda,Emel Müftüoğlu,Hovarda,1WYxdDka4CGsqcHtXPIpPx,0mJwTj7ldazo0CIYPt4wkN,0,2006,251868,False,https://open.spotify.com/track/0mJwTj7ldazo0CI...,...,0.682,6,-13.481,0,0.0928,0.0320,0.000000,0.0565,0.917,130.050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Gönlünü Gün Edeni,Mustafa Sandal,Devamı Var,6g0WD34QHSNrLMX9u9zHdh,0BMtYyexTw5GN1FB9zoAhY,0,2007-06-01,250293,False,https://open.spotify.com/track/0BMtYyexTw5GN1F...,...,0.744,11,-6.842,0,0.0307,0.2260,0.000549,0.2490,0.888,127.035
96,Bu Aşkın Katili Sensin,Fettah Can,Aklımda Kalanlar,3PHV8SBrlhbt06GFGtliDN,6HIeWAhjUPdKU8NXSHqlZP,44,2012-03-01,232339,False,https://open.spotify.com/track/6HIeWAhjUPdKU8N...,...,0.855,0,-4.779,0,0.1040,0.4620,0.000000,0.1540,0.776,92.006
97,Boş Bardak,Fettah Can,Aklımda Kalanlar,3PHV8SBrlhbt06GFGtliDN,2fFoXj2aUlaezPp3SinfSD,58,2012-03-01,226627,False,https://open.spotify.com/track/2fFoXj2aUlaezPp...,...,0.855,6,-6.671,0,0.0516,0.4420,0.000007,0.1050,0.799,96.016
98,Sen En Çok Aşksın,Fettah Can,Sen En Çok Aşksın,58wGB7x2oPTBxEPaul9oMw,3HOFotcK8kKVR7JxHzpJmV,40,2016-04-29,196821,False,https://open.spotify.com/track/3HOFotcK8kKVR7J...,...,0.783,5,-5.960,0,0.0308,0.1800,0.000000,0.2890,0.963,140.014


In [7]:
music_df.isnull().sum()

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
data = music_df

In [10]:
from datetime import datetime

# Yayın tarihine dayalı olarak ağırlıklı popülerlik puanlarını hesaplamak için bir fonksiyon
def calculate_weighted_popularity(release_date):
    # Yayın tarihini '%Y-%m-%d' biçiminden datetime nesnesine dönüştür
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Yayın yılını al
    release_year = release_date.year

    # Bugünkü yılı al
    current_year = datetime.now().year

    # Yıl farkına dayalı olarak ağırlıklı popülerlik puanını hesapla (örneğin, daha yeni yayınlar daha yüksek ağırlığa sahiptir)
    weight = 1 / (current_year - release_year + 1)
    return weight


In [11]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# Müzik özelliklerine dayalı içerik tabanlı öneriler almak için bir fonksiyon
def content_based_recommendations(input_song_name, num_recommendations=5):
    # Eğer giriş parça adı veri kümesinde bulunmuyorsa hata mesajı yazdır
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Müzik DataFrame içinde giriş parçasının indeksini al
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Müzik özelliklerine dayalı benzerlik puanlarını hesapla (kosinüs benzerliği)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # En benzer parçaların indekslerini al
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # İçerik tabanlı filtreleme temelinde en benzer parçaların adlarını al
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations


In [13]:
import pandas as pd

# Varsayılan olarak bir music_df DataFrame'i ve diğer gerekli fonksiyonların tanımlandığını varsayalım

# Ağırlıklı popülerliğe dayalı hibrit önerileri almak için bir fonksiyon
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    # Eğer giriş parça adı veri kümesinde bulunmuyorsa hata mesajı yazdır
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # İçerik tabanlı önerileri al
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Giriş parçasının popülerlik puanını al
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Ağırlıklı popülerlik puanını hesapla
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Ağırlıklı popülerliğe dayalı önerileri içerik tabanlı önerilere ekleyerek birleştir
    hybrid_recommendations = content_based_rec.copy()  # content_based_rec'in bir DataFrame olduğundan emin olun
    hybrid_recommendations.loc[len(hybrid_recommendations)] = {
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }

    # Ağırlıklı popülerlik puanına göre hibrit önerileri sırala
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Önerilerden giriş parçasını kaldır
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations


In [14]:

input_song_name = "Boş Bardak"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)



In [15]:
print(f"Hybrid recommended songs for '{input_song_name}':")
recommendations

Hybrid recommended songs for 'Boş Bardak':


,Track Name,Artists,Album Name,Release Date,Popularity
7,Bi Seni Konuşurum,Göksel,Arka Bahçem,2005-05-11,60.0
42,"Saz mı, Caz mı",Gülşen,Be Adam,1996-07-11,50.0
25,Oyunbaz,Yalın,Oyunbaz,2020-07-03,0.0
32,"Saz mı, Caz mı",Gülşen,Be Adam,1996-07-11,0.0
87,İşim Olmaz,Tarkan,Adımı Kalbine Yaz,2010-07-12,0.0
